# Notebook final del desarrollo del Proyecto

## 1. Importando el dataset de trabajo

In [102]:
import pandas as pd
import numpy as np
import datetime as dt

Se importa el dataset reducido a partir del original encontrado en Kaggle. El proceso de reducción se describe en el Readme.

In [103]:
df_model = pd.read_csv("../data/df_reducido.csv.gz")

In [104]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7603890 entries, 0 to 7603889
Data columns (total 9 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Marketing_Airline_Network  object 
 1   OriginCityName             object 
 2   DestCityName               object 
 3   Distance                   float64
 4   Month                      int64  
 5   DayofWeek                  int64  
 6   Holidays                   bool   
 7   Delayed                    int64  
 8   CRSDepTime                 int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 471.4+ MB


Se divide por tipos los features del dataset. Esto se hace pensando en el codificado de features que se hará más adelante.

## 2. Feature Engineering
Se dasarrolla la lógica del codificado de features.

In [105]:
TARGET = "Delayed"

CAT_COLS = [
    "Marketing_Airline_Network",
    "OriginCityName",
    "DestCityName"
]

CYCLIC_COLS = {
    "DayofWeek": 7,
    "Month": 12,
    "CRSDepTime": 1440
}

NUM_COLS = ["Distance"]
BOOL_COLS = ["Holidays"]

In [106]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))

Importación de encoders trabajados originalmente en el notebook, pero externalizados posteriormente y colocados en la carpeta src.

In [107]:
from src.encoders import (
    CyclicEncoder,
    TargetEncoder,
    CAT_COLS,
    CYCLIC_COLS
)

Código para codificar las variables ciclicas (Month, Dayofweek y CRSDepTime), externalizado. Se dejó comentado en el notebook.
El fin de esta codificación es que el modelo entienda sobre periodos circulares, es decir, entiende que Domingo y Lunes están cerca así como Diciembre y Enero. Originalmente, en ambos casos se entienden como si fueran extremos.

In [64]:
"""
def add_cyclic_features(df: pd.DataFrame, cyclic_cols: dict) -> pd.DataFrame:
    df = df.copy()

    for col, period in cyclic_cols.items():
        df[f"{col}_sin"] = np.sin(2 * np.pi * df[col] / period)
        df[f"{col}_cos"] = np.cos(2 * np.pi * df[col] / period)

    df = df.drop(columns=list(cyclic_cols.keys()))
    return df
"""

In [65]:
"""
from sklearn.base import BaseEstimator, TransformerMixin

class CyclicEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cyclic_cols: dict):
        self.cyclic_cols = cyclic_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return add_cyclic_features(X, self.cyclic_cols)
"""

El dataset reducido extraemos 500K estratificando el TARGET (Delayed). Nos valemos de la biblioteca sklearn.

In [108]:
from sklearn.model_selection import train_test_split

df_work, _ = train_test_split(
    df_model,
    train_size=500_000,
    stratify=df_model[TARGET],
    random_state=42
)

Preparamos el dataset para el entrenamiento del modelo.

In [109]:
X = df_work.drop(columns=[TARGET])
y = df_work[TARGET]

In [110]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [111]:
from sklearn.base import BaseEstimator, TransformerMixin

Código para aplicar Target Encoding a variables categóricas, implementado como un transformer compatible con scikit-learn. También externalizado y comentado en notebook.
Se reemplaza cada categoría por la media del target asociada a ella, suavizada con la media global para evitar sobreajuste en categorías con pocos datos. El smoothing controla cuánto peso se le da a la estadística propia de la categoría frente al promedio global. 
El objetivo es transformar variables categóricas de alta cardinalidad (son cientos de ciudades) en valores numéricos informativos, reduciendo ruido y mejorando la capacidad predictiva del modelo sin introducir relaciones artificiales entre categorías.

In [113]:
"""
class TargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols, smoothing=20):
        self.cols = cols
        self.smoothing = smoothing

    def fit(self, X, y):
        X = X.copy()
        y = y.copy()

        self.global_mean_ = y.mean()
        self.encoding_ = {}

        for col in self.cols:
            stats = (
                pd.concat([X[col], y], axis=1)
                .groupby(col)[y.name]
                .agg(["mean", "count"])
            )

            smooth = (
                (stats["count"] * stats["mean"] +
                 self.smoothing * self.global_mean_)
                / (stats["count"] + self.smoothing)
            )

            self.encoding_[col] = smooth

        return self

    def transform(self, X):
        X = X.copy()

        for col in self.cols:
            X[col] = (
                X[col]
                .map(self.encoding_[col])
                .fillna(self.global_mean_)
            )

        return X
"""

'\nclass TargetEncoder(BaseEstimator, TransformerMixin):\n    def __init__(self, cols, smoothing=20):\n        self.cols = cols\n        self.smoothing = smoothing\n\n    def fit(self, X, y):\n        X = X.copy()\n        y = y.copy()\n\n        self.global_mean_ = y.mean()\n        self.encoding_ = {}\n\n        for col in self.cols:\n            stats = (\n                pd.concat([X[col], y], axis=1)\n                .groupby(col)[y.name]\n                .agg(["mean", "count"])\n            )\n\n            smooth = (\n                (stats["count"] * stats["mean"] +\n                 self.smoothing * self.global_mean_)\n                / (stats["count"] + self.smoothing)\n            )\n\n            self.encoding_[col] = smooth\n\n        return self\n\n    def transform(self, X):\n        X = X.copy()\n\n        for col in self.cols:\n            X[col] = (\n                X[col]\n                .map(self.encoding_[col])\n                .fillna(self.global_mean_)\n        

Se entrena el baseline dummy que tiene una precisión de 0.76, lo cual refleja la estadística aproximada de que los vuelos normalmente se retrasan en el orden aproximado del 80%. No obstante, no predice ningún vuelo con retraso. Como se puede observar la precisión para la categoría 1, que es de retraso, es cero así como el recall es cero. Es porque este dummy siempre dice "puntual" y por simple estadistica, acierta 76% de las veces, pero NO DETECTA NINGÚN RETRASO.

## 3. Entrenamiento del modelo

In [126]:
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

dummy_pipeline = Pipeline(
    steps=[
        ("cyclic", CyclicEncoder(CYCLIC_COLS)),
        ("target_enc", TargetEncoder(cols=CAT_COLS, smoothing=20)),
        ("model", DummyClassifier(strategy="most_frequent"))
    ]
)

dummy_pipeline.fit(X_train, y_train)

y_pred_dummy = dummy_pipeline.predict(X_test)

print("=== Dummy Classifier (Pipeline) ===")
print(classification_report(y_test, y_pred_dummy))

=== Dummy Classifier (Pipeline) ===
              precision    recall  f1-score   support

           0       0.76      1.00      0.86     75521
           1       0.00      0.00      0.00     24479

    accuracy                           0.76    100000
   macro avg       0.38      0.50      0.43    100000
weighted avg       0.57      0.76      0.65    100000



C:\Projects\flight_delay_api\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Projects\flight_delay_api\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Projects\flight_delay_api\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Se elige para el proyecto el modelo RandomForest, con un treshold de 0.40. Se puede observar que el recall de la categoría 1 es 0.78, es decir, el modelo predice 4 de cada 5 retrasos. La accuracy baja nos revela que el modeo predecirá como retrasos varios vueltos puntuales. Sin embargo, se tolera con el fin es tener a los clientes mejor informados en caso de una probabilidad de retraso.

In [115]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

rf_pipeline = Pipeline(
    steps=[
        ("cyclic", CyclicEncoder(CYCLIC_COLS)),
        ("target_enc", TargetEncoder(cols=CAT_COLS, smoothing=20)),
        ("model", RandomForestClassifier(
            n_estimators=300,
            max_depth=20,
            min_samples_leaf=50,
            class_weight={0: 1, 1: 3},
            n_jobs=-1,
            random_state=42
        ))
    ]
)

rf_pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('cyclic', ...), ('target_enc', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,cyclic_cols,"{'CRSDepTime': 1440, 'DayofWeek': 7, 'Month': 12}"
,cols,"['Marketing_Airline_Network', 'OriginCityName', ...]"
,smoothing,20
,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",300
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",20
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2


In [116]:
y_prob_rf = rf_pipeline.predict_proba(X_test)[:, 1]

threshold = 0.40
y_pred_rf = (y_prob_rf >= threshold).astype(int)

print(f"=== RandomForest (Pipeline) | threshold={threshold} ===")
print(classification_report(y_test, y_pred_rf))

=== RandomForest (Pipeline) | threshold=0.4 ===
              precision    recall  f1-score   support

           0       0.87      0.46      0.60     75521
           1       0.32      0.78      0.45     24479

    accuracy                           0.54    100000
   macro avg       0.59      0.62      0.53    100000
weighted avg       0.73      0.54      0.56    100000



## 4. Exportación del modelo y pruebas finales

In [118]:
import joblib

MODEL_PATH = "flight_delay_pipeline.joblib"

joblib.dump(rf_pipeline, MODEL_PATH)

['flight_delay_pipeline.joblib']

In [119]:
loaded_pipeline = joblib.load(MODEL_PATH)

y_prob_loaded = loaded_pipeline.predict_proba(X_test)[:, 1]
y_pred_loaded = (y_prob_loaded >= 0.4).astype(int)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_loaded))

              precision    recall  f1-score   support

           0       0.87      0.46      0.60     75521
           1       0.32      0.78      0.45     24479

    accuracy                           0.54    100000
   macro avg       0.59      0.62      0.53    100000
weighted avg       0.73      0.54      0.56    100000



In [120]:
import pandas as pd
import holidays

# Define país una sola vez (esto es configuración, no lógica)
HOLIDAYS_CALENDAR = holidays.US()


def extract_features(payload: dict) -> pd.DataFrame:
    """
    Convierte un JSON de entrada en un DataFrame
    compatible con el pipeline entrenado.
    """

    # --- Validaciones mínimas ---
    required_keys = {
        "aerolinea",
        "origen",
        "destino",
        "fecha_partida",
        "distancia_km",
    }

    missing = required_keys - payload.keys()
    if missing:
        raise ValueError(f"Faltan campos obligatorios: {missing}")

    # --- Parseo de fecha ---
    dt = pd.to_datetime(payload["fecha_partida"], errors="raise")

    features = {
        "Marketing_Airline_Network": payload["aerolinea"],
        "OriginCityName": payload["origen"],
        "DestCityName": payload["destino"],
        "Distance": float(payload["distancia_km"]),
        "Month": dt.month,
        "DayofWeek": dt.dayofweek,          # 0 = lunes
        "CRSDepTime": dt.hour * 60 + dt.minute,  # minutos desde medianoche
        "Holidays": dt.date() in HOLIDAYS_CALENDAR,
    }

    return pd.DataFrame([features])

In [121]:
payload_test = {
    "aerolinea": "AA",
    "origen": "New York, NY",
    "destino": "Los Angeles, CA",
    "fecha_partida": "2025-11-10T14:30:00",
    "distancia_km": 3983
}

X_api = extract_features(payload_test)

In [122]:
X_api

,Marketing_Airline_Network,OriginCityName,DestCityName,Distance,Month,DayofWeek,CRSDepTime,Holidays
0,AA,"New York, NY","Los Angeles, CA",3983.0,11,0,870,False


In [123]:
pipeline = joblib.load(MODEL_PATH)

In [124]:
THRESHOLD = 0.40  # decisión de negocio


def predict_delay(payload: dict) -> dict:
    """
    Recibe un JSON de vuelo y devuelve
    la predicción de retraso y su probabilidad.
    """

    # 1️⃣ JSON → DataFrame
    X = extract_features(payload)

    # 2️⃣ Probabilidad de retraso (clase 1)
    prob_delay = pipeline.predict_proba(X)[0, 1]

    # 3️⃣ Decisión final
    prediction = int(prob_delay >= THRESHOLD)

    # 4️⃣ Respuesta amigable
    return {
        "prevision": "Retrasado" if prediction == 1 else "Puntual",
        "probabilidad": round(float(prob_delay), 2),
    }

In [125]:
payload_test = {
    "aerolinea": "AA",
    "origen": "New York, NY",
    "destino": "Los Angeles, CA",
    "fecha_partida": "2025-11-10T14:30:00",
    "distancia_km": 3983
}

result = predict_delay(payload_test)
print(result)

{'prevision': 'Retrasado', 'probabilidad': 0.49}
